In [1]:
%matplotlib inline

import sys
import os
import numpy as np
import json
import pandas as pd
import traceback

sys.path.append("../../../") # add results folder as a python path 
from viz import *
from analyze import *

In [2]:
path = os.path.dirname("./data2/BOHB/iter30_result/")
target_dirs = os.listdir(path)

In [3]:
def getTrialResult(result):
    cum_exec_time = []
    cum_opt_time = []
    cum_first = True

    exec_times = []
    opt_times = []
    epochs = []
    model_info = []
    errors = []
    accuracies = []

    for r in result:
        dict_r = json.loads(r)
#         print(dict_r)
        exec_time = dict_r[2]['finished'] - dict_r[2]['started']
        opt_time = dict_r[2]['finished'] - dict_r[2]['submitted']

        if cum_first is True:
            cum_exec_time_last_value = 0.0
            cum_opt_time_last_value = 0.0
            cum_first = False

        if cum_exec_time_last_value <= 86400:
            exec_times.append(exec_time)
            opt_times.append(opt_time)
            cum_exec_time.append((cum_exec_time_last_value + exec_time))
            cum_opt_time.append((cum_opt_time_last_value + opt_time))
            epochs.append(dict_r[1])
            model_info.append(dict_r[0])
            if dict_r[3] is not None:
                errors.append(dict_r[3]['loss'])
                accuracies.append(dict_r[3]['info']['test accuracy'])
            else:
                errors.append(1.0)
                accuracies.append(0.0)
            
            cum_exec_time_last_value = cum_exec_time[len(cum_exec_time)-1]
            cum_opt_time_last_value = cum_opt_time[len(cum_opt_time)-1]
            
            dict_result = dict([("cum_exec_time", cum_exec_time),
                        ("exec_time", exec_times),
                        ("opt_time", opt_times),
                        ("model_info", model_info),
                        ("cum_opt_time", cum_opt_time),
                        ("error", errors),
                        ("accuracy", accuracies),
                        ("epoch", epochs)])
        else:
            continue
        
    return dict_result

In [4]:
def getTrialConfig(config):
    config_dict = dict()
    
    for c in config:
        dict_c = json.loads(c)
        target = dict_c[2]
        config_dict[str(dict_c[0])] = target
        
    return config_dict

In [5]:
def addModelInfo(trial_result, trial_config):
    model_index_list = []
    for i in range(len(trial_result['model_info'])):
        model_idx = str(trial_result['model_info'][i])
        model_info = trial_config[model_idx]
        target_index = -2
        if 'lookup_index' in model_info:
            target_index = model_info['lookup_index']
        elif trial_result['error'][i] == 1:
            target_index = 0
        model_index_list.append(target_index)
    trial_result['model_index'] = model_index_list
    del trial_result['model_info']

In [6]:
bohb_results = dict()
bohb_configs = dict()

for d in target_dirs:
# #     print(d)
#     if d != 'iter10_result':
#         continue
#     else:
#         trial_num = '0'
# #     trial_num = d[5:]
    trial_num = d[11:]
    
    with open(path+'/'+d+'/'+'results.json') as rf:
        result = rf.readlines()

    result = [x.strip() for x in result]
    result_dict=getTrialResult(result)
    
    with open(path+'/'+d+'/'+'configs.json') as rf:
        config = rf.readlines()
        
    config = [x.strip() for x in config]
#     bohb_configs[trial_num]=getTrialConfig(config)
    config_dict=getTrialConfig(config)
    
    addModelInfo(result_dict, config_dict)
    
    bohb_results[trial_num]=result_dict
    

In [7]:
with open('iter30_result.json', 'w') as outfile:
    json.dump(bohb_results, outfile)

In [10]:
bohb_results['0']

{'cum_exec_time': [16.559516191482544,
  27.73139524459839,
  39.08903431892395,
  39.11551117897034,
  50.32280087471008,
  50.34768533706665,
  61.643877267837524,
  72.88218450546265,
  84.38546919822693,
  123.09992527961731,
  161.5787193775177,
  200.63519310951233,
  308.66562962532043,
  308.6895272731781,
  347.7972536087036,
  347.82300424575806,
  463.73482871055603,
  587.8084886074066,
  710.2897763252258,
  710.3178799152374,
  722.8149073123932,
  722.8449578285217,
  735.3548262119293,
  735.5187287330627,
  748.0230412483215,
  748.0516035556793,
  760.5659739971161,
  760.5929002761841,
  773.0408833026886,
  816.7252721786499,
  860.4759500026703,
  904.4764342308044,
  1026.8273830413818,
  1070.7757380008698,
  1070.801866054535,
  1070.8298137187958,
  1193.0627162456512,
  1193.0898787975311,
  1193.1190395355225,
  1314.8233859539032,
  1327.3421375751495,
  1339.892682313919,
  1352.6391360759735,
  1365.1723330020905,
  1377.5955936908722,
  1390.0390105247498